In [1]:
import tensorflow as tf
import h5py
#import pickle
import dask
import dask.array as da
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/classification/cifar_10/data/test_y')
train_x, train_y, test_x, test_y

(dask.array<from-npy-stack, shape=(50000, 32, 32, 3), dtype=float64, chunksize=(10000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(50000, 1), dtype=float64, chunksize=(10000, 1)>,
 dask.array<from-npy-stack, shape=(10000, 32, 32, 3), dtype=float64, chunksize=(1000, 32, 32, 3)>,
 dask.array<from-npy-stack, shape=(10000, 1), dtype=float64, chunksize=(1000, 1)>)

In [3]:
class CIFAR10Sequence(tf.keras.utils.Sequence):

    def __init__(self, x, y, batch_size):
        self.x, self.y = x, y
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    '''
    def on_epoch_end(self):
        # Shuffles indexes after each epoch
        self.indexes = da.random.choice(len(self.x), len(self.x), False, chunks=len(self.x))
    '''

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        #return dask.compute(batch_x, batch_y)
        return batch_x, batch_y
    
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    vertical_flip=True,
    horizontal_flip=True)

def build_block(input_layer, filters, norm=True, k=[3,3]):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=(k[0], k[1]), padding='same', use_bias=False, kernel_initializer='glorot_uniform')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_model(num_class):
    image_input = tf.keras.Input(shape=(32, 32, 3), name='input_layer')
    conv_1 = build_block(image_input, 48)
    conv_2 = build_block(conv_1, 48)
    pool_1 = tf.keras.layers.MaxPooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.Dropout(0.6)(pool_1)
    conv_3 = build_block(drop_1, 96)
    conv_4 = build_block(conv_3, 96)
    pool_2 = tf.keras.layers.MaxPooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.Dropout(0.6)(pool_2)
    conv_5 = build_block(drop_2, 192)
    conv_6 = build_block(conv_5, 192)
    pool_3 = tf.keras.layers.MaxPooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.Dropout(0.6)(pool_3)
    conv_7 = build_block(drop_3, 192, False, [1,1])
    gap = tf.keras.layers.GlobalAvgPool2D()(conv_7)
    logits = tf.keras.layers.Dense(units=num_class, activation='softmax', bias_initializer='ones', kernel_initializer='glorot_uniform')(gap)
    model = tf.keras.Model(inputs=image_input, outputs=logits)
    return model

In [4]:
model = build_model(10)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 48)        1296      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 48)        192       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 48)        20736     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 48)        192       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 48)        0         
__________

In [5]:
model.load_weights("weights_d20.best.hdf5")
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
scores = model.evaluate(test_x, test_y)
print(scores)

10000/10000 [==============================] - 19s 2ms/step
[0.47553113974928857, 0.8812]


In [6]:
epochs, learning_rate, batch_size = 10000, 0.001, 128

#train_generator = CIFAR10Sequence(train_x, train_y, batch_size)
test_generator = CIFAR10Sequence(test_x, test_y, batch_size)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(train_x)

filepath="weights_d21.best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.load_weights(filepath)
# fits the model on batches with real-time data augmentation:
model.fit_generator(generator=datagen.flow(train_x, train_y, batch_size=batch_size),
                    steps_per_epoch=(len(train_x) // batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=(len(test_x) // batch_size),
                    use_multiprocessing=True,
                    workers=12,
                    shuffle=True,
                    callbacks=[checkpoint])


Epoch 1/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2757 - acc: 0.9019
Epoch 00001: acc improved from -inf to 0.90199, saving model to weights_d20.best.hdf5
390/390 [==============================] - 40s 102ms/step - loss: 0.2757 - acc: 0.9019 - val_loss: 0.4931 - val_acc: 0.8764
Epoch 2/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2748 - acc: 0.9039
Epoch 00002: acc improved from 0.90199 to 0.90397, saving model to weights_d20.best.hdf5
390/390 [==============================] - 34s 88ms/step - loss: 0.2746 - acc: 0.9040 - val_loss: 0.5161 - val_acc: 0.8725
Epoch 3/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2740 - acc: 0.9035
Epoch 00003: acc did not improve from 0.90397
390/390 [==============================] - 34s 87ms/step - loss: 0.2739 - acc: 0.9036 - val_loss: 0.4445 - val_acc: 0.8842
Epoch 4/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2731 - acc: 0.9036
Epoch 00004: acc did not improve 

Epoch 32/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2718 - acc: 0.9031
Epoch 00032: acc did not improve from 0.90578
390/390 [==============================] - 36s 91ms/step - loss: 0.2718 - acc: 0.9030 - val_loss: 0.4927 - val_acc: 0.8759
Epoch 33/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2789 - acc: 0.9018
Epoch 00033: acc did not improve from 0.90578
390/390 [==============================] - 36s 91ms/step - loss: 0.2788 - acc: 0.9019 - val_loss: 0.4792 - val_acc: 0.8796
Epoch 34/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2729 - acc: 0.9021
Epoch 00034: acc did not improve from 0.90578
390/390 [==============================] - 36s 92ms/step - loss: 0.2731 - acc: 0.9021 - val_loss: 0.4385 - val_acc: 0.8863
Epoch 35/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2751 - acc: 0.9031
Epoch 00035: acc did not improve from 0.90578
390/390 [==============================] - 36s 92ms/step - loss: 0.

389/390 [============================>.] - ETA: 0s - loss: 0.2733 - acc: 0.9037
Epoch 00063: acc did not improve from 0.90578
390/390 [==============================] - 36s 92ms/step - loss: 0.2734 - acc: 0.9037 - val_loss: 0.4641 - val_acc: 0.8819
Epoch 64/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2778 - acc: 0.9037
Epoch 00064: acc did not improve from 0.90578
390/390 [==============================] - 36s 92ms/step - loss: 0.2777 - acc: 0.9036 - val_loss: 0.4120 - val_acc: 0.8913
Epoch 65/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2744 - acc: 0.9034
Epoch 00065: acc did not improve from 0.90578
390/390 [==============================] - 36s 91ms/step - loss: 0.2742 - acc: 0.9035 - val_loss: 0.4728 - val_acc: 0.8830
Epoch 66/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2744 - acc: 0.9030
Epoch 00066: acc did not improve from 0.90578
390/390 [==============================] - 36s 92ms/step - loss: 0.2743 - acc: 0.9

Epoch 94/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2710 - acc: 0.9031
Epoch 00094: acc did not improve from 0.90632
390/390 [==============================] - 36s 92ms/step - loss: 0.2715 - acc: 0.9030 - val_loss: 0.5200 - val_acc: 0.8700
Epoch 95/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2669 - acc: 0.9037
Epoch 00095: acc did not improve from 0.90632
390/390 [==============================] - 36s 91ms/step - loss: 0.2670 - acc: 0.9037 - val_loss: 0.4981 - val_acc: 0.8740
Epoch 96/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2725 - acc: 0.9036
Epoch 00096: acc did not improve from 0.90632
390/390 [==============================] - 36s 92ms/step - loss: 0.2725 - acc: 0.9035 - val_loss: 0.5342 - val_acc: 0.8743
Epoch 97/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2721 - acc: 0.9033
Epoch 00097: acc did not improve from 0.90632
390/390 [==============================] - 36s 91ms/step - loss: 0.

Epoch 125/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2707 - acc: 0.9043
Epoch 00125: acc did not improve from 0.90632
390/390 [==============================] - 36s 92ms/step - loss: 0.2706 - acc: 0.9044 - val_loss: 0.4199 - val_acc: 0.8909
Epoch 126/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2770 - acc: 0.9015
Epoch 00126: acc did not improve from 0.90632
390/390 [==============================] - 36s 91ms/step - loss: 0.2770 - acc: 0.9016 - val_loss: 0.4536 - val_acc: 0.8843
Epoch 127/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2742 - acc: 0.9049
Epoch 00127: acc did not improve from 0.90632
390/390 [==============================] - 36s 92ms/step - loss: 0.2742 - acc: 0.9048 - val_loss: 0.4582 - val_acc: 0.8859
Epoch 128/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2738 - acc: 0.9024
Epoch 00128: acc did not improve from 0.90632
390/390 [==============================] - 36s 91ms/step - loss

...

Epoch 1582/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2712 - acc: 0.9030
Epoch 01582: acc did not improve from 0.90762
390/390 [==============================] - 37s 94ms/step - loss: 0.2713 - acc: 0.9030 - val_loss: 0.4481 - val_acc: 0.8842
Epoch 1583/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2621 - acc: 0.9063
Epoch 01583: acc did not improve from 0.90762
390/390 [==============================] - 36s 93ms/step - loss: 0.2619 - acc: 0.9064 - val_loss: 0.4233 - val_acc: 0.8917
Epoch 1584/10000
389/390 [============================>.] - ETA: 0s - loss: 0.2685 - acc: 0.9037
Epoch 01584: acc did not improve from 0.90762
390/390 [==============================] - 36s 93ms/step - loss: 0.2688 - acc: 0.9036 - val_loss: 0.4622 - val_acc: 0.8823
Epoch 1585/10000
262/390 [===================>..........] - ETA: 11s - loss: 0.2719 - acc: 0.9034

Process ForkPoolWorker-37978:
Process ForkPoolWorker-37973:
Process ForkPoolWorker-37972:
Process ForkPoolWorker-37977:
Process ForkPoolWorker-37974:
Process ForkPoolWorker-37970:
Process ForkPoolWorker-37979:
Process ForkPoolWorker-37980:
Process ForkPoolWorker-37969:
Process ForkPoolWorker-37976:
Process ForkPoolWorker-37971:
Process ForkPoolWorker-37975:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/proc

  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 171, in fit_generator
    x, y, sample_weight=sample_weight, class_weight=class_weight)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1828, in train_on_batch
    outputs = self.train_function(ins)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/keras/backend.py", line 2978, in __call__
    run_metadata=self.run_metadata)
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1399, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skyolia/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/s

TypeError: must be str, not list

In [9]:
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(train_x, train_y, batch_size=32):
        print(type(train_x))
        print(type(x_batch))

Epoch 0
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class '

<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.ndarray'>
<class 'dask.array.core.Array'>
<class 'numpy.nd

KeyboardInterrupt: 